# Exploración Inicial

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [1]:
import pandas as pd
import pickle

import pandas as pd
import dask.dataframe as dd
import time

import scipy.stats
import re

## Obtencion de AGEBS de interés:

In [2]:
df_estaciones = pd.read_csv("/Users/danielbustillos/Documents/ecobici_viz/data/SIG/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv")
df_estaciones.head()

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
0,440,440 REAL DE MAYORAZGO-PUENTE XOCO,440 - Real De Mayorazgo-Puente Xoco,S/N,3330.0,1,Ampliación Granada,NaN,433,19.360650,...,NaN,NaN,NaN,"19.36065,-99.168669",0967,14,1,9,4550,Movement Zone 4550
1,283,283 AV. COYOACÁN-RAFAEL DONDE,283 - Av. Coyoacán-Rafael Donde,S/N,3103.0,1,Ampliación Granada,NaN,279,19.398270,...,NaN,NaN,NaN,"19.39827,-99.167383",0045,14,1,9,4551,Movement Zone 4551
2,279,279 LA MORENA-PROVIDENCIA,279 - La Morena-Providencia,S/N,3103.0,DVN,Del Valle Norte,NaN,164,19.399908,...,NaN,NaN,NaN,"19.399908,-99.166553",0045,14,1,9,4551,Movement Zone 4551
3,443,443 BRUNO TRAVEN-GOLONDRINAS,443 - Bruno Trave-Avenida México-Coyoacan,S/N,3340.0,1,Ampliación Granada,NaN,432,19.359655,...,NaN,NaN,NaN,"19.359655,-99.162025",0971,14,1,9,4553,Movement Zone 4553
4,432,432 CARRILLO PUERTO-AV. MÉXICO-COYOACÁN,432 - Carrillo Pto-Golondrinas,S/N,3340.0,GPM,General Pedro Maria Anaya,NaN,442,19.361296,...,NaN,NaN,NaN,"19.361296,-99.162581",0971,14,1,9,4553,Movement Zone 4553


In [3]:
df_estaciones['CVE_AGEB'] = [''.join(filter(lambda x: x.isdigit(), row)) for row in df_estaciones['CVE_AGEB']]
df_estaciones['CVE_AGEB'] = df_estaciones['CVE_AGEB'].astype(int)

In [4]:
lista_agebs = df_estaciones['CVE_AGEB'].astype(int).unique().tolist()
len(lista_agebs)

143

In [5]:
lista_agebs[:5]

[967, 45, 971, 219, 187]

Guardamos lo AGEBS en una lista:

In [6]:
with open("../data/lista_agebs.txt", "wb") as fp:
    pickle.dump(lista_agebs, fp)

## Leamos el archivo de viajes de UBER

In [7]:
path_uber_trips = "../data/uber/mexico_city-agebs-2019-1-OnlyWeekdays-HourlyAggregate.csv"

In [8]:
## cambiar a pandas

In [9]:
t=time.clock()
df_trips = dd.read_csv(path_uber_trips)
df_trips = df_trips.compute()
print("load train: " , time.clock()-t)

load train:  49.177875


In [10]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,1599,1710,4,629.46,600.50,498.86,1.80
1,3277,3083,8,1651.38,672.78,1549.81,1.40
2,977,3047,6,1165.89,268.38,1137.46,1.25
3,963,3187,6,931.03,284.51,898.61,1.29
4,3758,4915,13,2632.10,510.63,2583.73,1.21


In [11]:
list(set(lista_agebs) - set(df_trips["sourceid"].unique()))

[1356, 717, 783, 1360, 49, 401, 533, 1337, 1341, 1214]

In [12]:
df_trips = df_trips[df_trips["sourceid"].isin(lista_agebs)]

In [13]:
#df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [14]:
df_trips.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [15]:
df_trips.reset_index(drop=True, inplace=True)

In [16]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,806,4757,6,4167.13,913.33,4073.42,1.24
1,967,3147,6,1675.38,472.15,1613.91,1.31
2,803,4787,6,4408.13,1432.03,4032.96,1.63
3,1375,292,22,1508.99,364.23,1469.56,1.26
4,1430,4945,5,1107.10,227.94,1090.38,1.18


In [17]:
df_trips["sourceid"].nunique()

133

### tiempo máximo y mínimos por viaje y hora.
Suponemos una distribución normal con media 'mean_travel_time' y desviación estandar 'standard_deviation_travel_time>'

In [18]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,806,4757,6,4167.13,913.33,4073.42,1.24
1,967,3147,6,1675.38,472.15,1613.91,1.31
2,803,4787,6,4408.13,1432.03,4032.96,1.63
3,1375,292,22,1508.99,364.23,1469.56,1.26
4,1430,4945,5,1107.10,227.94,1090.38,1.18


In [19]:
def travel_min(mean,std):
    min = scipy.stats.norm(mean, std).ppf(.10)
    #max = scipy.stats.norm(mean, std).ppf(.90)
    return(min)
def travel_max(mean,std):
    #min = scipy.stats.norm(mean, std).ppf(.10)
    max = scipy.stats.norm(mean, std).ppf(.90)
    return(max)

In [20]:
df_trips['min_travel_time'] = 0
df_trips['max_travel_time'] = 0

In [21]:
df_trips['min_travel_time'] =  df_trips.apply(
    lambda row : travel_min(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [22]:
df_trips['max_travel_time'] =  df_trips.apply(
    lambda row : travel_max(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [23]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,min_travel_time,max_travel_time
0,806,4757,6,4167.13,913.33,4073.42,1.24,2996.650509,5337.609491
1,967,3147,6,1675.38,472.15,1613.91,1.31,1070.295428,2280.464572
2,803,4787,6,4408.13,1432.03,4032.96,1.63,2572.909712,6243.350288
3,1375,292,22,1508.99,364.23,1469.56,1.26,1042.210473,1975.769527
4,1430,4945,5,1107.10,227.94,1090.38,1.18,814.983136,1399.216864


## Definimos periodos del día para non graficar todas las horas:

In [24]:
def periodo_hora(row):
    if row in [7,8,9]:
        return "7-10"
    elif row in[10,11,12]:
        return "10-13"
    elif row in [13,14,15]:
        return "13-16"
    elif row in [16,17,18]:
        return "16-19"
    elif row in [19,20,21]:
        return "19-22"
    elif row in [22,23,0]:
        return "22-1"
    else:
        return None

df_trips["Hora_Retiro_bin"] = df_trips["hod"].apply(periodo_hora)

df_trips = df_trips.dropna()

## Convertimos la duración de los viajes de segundos a minutos:

In [28]:
df_trips['max_travel_time']  = df_trips['max_travel_time'] / 60
df_trips['min_travel_time']  = df_trips['min_travel_time'] / 60
df_trips['mean_travel_time']  = df_trips['mean_travel_time'] / 60

In [30]:
df_trips.columns

Index(['sourceid', 'dstid', 'hod', 'mean_travel_time',
       'standard_deviation_travel_time', 'geometric_mean_travel_time',
       'geometric_standard_deviation_travel_time', 'min_travel_time',
       'max_travel_time', 'Hora_Retiro_bin'],
      dtype='object')

seleccionamos columnnas

In [31]:
df_trips = df_trips[['sourceid', 'dstid', 'hod', 'mean_travel_time', 'min_travel_time',
       'max_travel_time', 'Hora_Retiro_bin']]

In [33]:
df_trips.to_csv("../data/production_data/viajes_uber.csv", index=False)